# Agents

In [2]:
import os
import langchain
import openai
import json

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

## Prompts

In [27]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [28]:
# Example of what "PromptTemplate" is doing
print(prompt_template.format(query="What is Doge"))

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: What is Doge

Answer: 


Pass prompt to model with a query not part of the 'Context.' The model should not know anything about it because we indicated so in the "Instructions" (first paragraph)

In [29]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

llm_openai(prompt_template.format(query="What is Doge"))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


"I don't know."

Now pass a query asking for something found in the context section. The model should be able to provide an answer

In [30]:
print(llm_openai(prompt_template.format(query="Which libraries and model providers offer LLMs?")))

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs.


Now let's compare that to simply calling the LLM without the instructions and the context. The LLM should use its own training wealth of information to provide a thorough answer. This can be useful in various occasions, but for bespoke purposes we may want to restrict, like the above

In [31]:
print(llm_openai("Which libraries and model providers offer LLMs?"))

1. Libraries:

   - Hugging Face: This library provides a wide range of pre-trained models for various NLP tasks, including language modeling. It supports several transformer-based models such as BERT, GPT-2, RoBERTa, and T5.

   - TensorFlow and Keras: These libraries provide functionalities to build and train language models from scratch. They also support pre-trained models.

   - PyTorch: Similar to TensorFlow, PyTorch also provides functionalities to build and train language models. It also supports pre-trained models.

   - NLTK: The Natural Language Toolkit (NLTK) is a Python library for symbolic and statistical natural language processing. It provides easy-to-use interfaces to over 50 corpora and lexical resources.

2. Model Providers:

   - OpenAI: They provide models like GPT-2 and GPT-3 which are state-of-the-art language models.

   - Google AI: They provide models like BERT and T5 which are widely used for various NLP tasks.

   - Facebook AI: They provide models like RoBE

Let's take a look at few shot examples. Few shot prompt templates is useful for source knowledge, that is, knowledge provided to the model at inference time. Few shot learning will train the model to understand what is it we are trying to do

In [32]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

llm_openai.model_kwargs["temperature"] = 1.0  # increase creativity/randomness of output

print(llm_openai(prompt))

Well, we could dive into deep philosophy and talk about existentialism and the human condition. Or we could go the route of the Hitchhiker's Guide to the Galaxy and say it's '42', or you know, we could say it's 'Pizza'. All seem equally valid!


<b>FewShotPromptTemplate</b> We set the temperature to 1 which should have increased its creativity and wittiness, but we didn'g really get that. We'll train it so that it can achieve that

In [33]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(llm_openai(prompt))

Life is like a box of chocolates. You never know what you're going to get.


It understood the assignment, literally. Now let's use langchain's formalized way of doing this FewShotPromptTemplate

In [34]:
from langchain import FewShotPromptTemplate

In [43]:
# create examples
examples = [
    {
        "query":"How are you?",
        "answer":"Who knows, I don't have energy to do most things, nothing interests me and I have trouble sleeping"
    },
    {
        "query":"What time is it?",
        "answer":"It's probably time to regret the past"
    }
]

# define template to use examples
example_template = """
User: {query}
AI: {answer}
"""

# create prompt to take in template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now create prefix and suffix that will wrap around the example prompt. The suffix will take in the User's query
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 
"""
suffix = """
User: {query}
AI: """

# now put it all together in our fewshot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
) 

# Now visualize it
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))


The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: Who knows, I don't have energy to do most things, nothing interests me and I have trouble sleeping


User: What time is it?
AI: It's probably time to regret the past


User: What is the meaning of life?
AI: 


Now test the fewshotprompt, it should output answers having seen the reinforced instructions. In our case, these reinforced instructions represent a bad mood, depressed assistant 

In [45]:
llm_openai(few_shot_prompt_template.format(query=query))

"Life? Let me think... It's just a series of disappointments and futile struggles, don't you think?"

Excellent, the AI understood from our fewshotprompt instructions that it needed to be depressed. Below is a regular answer for constrast

In [46]:
print(llm_openai(query))

As an artificial intelligence, I don't have personal beliefs or experiences, but many people believe that the meaning of life is a philosophical and metaphysical question closely tied to various sets of beliefs. Different people, religions, and philosophies suggest different answers to this question, such as happiness, love, pleasure, personal growth, service to others, or fulfillment of potential. Others suggest that life may have no intrinsic meaning, and it's up to each individual to create their own purpose and meaning.


<b>Why is this important?</b> We can certainly do all of this with f'string or dictionaries, but we might lose a benefit in langchain to maximize tokens and resources when prompting instructions. There is a functionality that <b>selects examples based on length need </b> which will allow to only use not all but examples matching lenght criteria. Since prompt windows have limits (ie. input + output), this will help maximize the usage and ensure appropriate choice when prompting. Let's check the exampel below

In [47]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]


In [49]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50 # No example should be beyond this length. This is the number of words, including newlines
)

In [57]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"], #user input variable
    example_separator="\n"
)

Let's test how this works with shorter queries. This should maximize the number of examples seen by the AI

In [58]:
print(dynamic_prompt_template.format(query="Do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: Do birds fly?
AI: 


This should then minimize the examples seen by the AI with longer queries

In [59]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


#### This covers a way to prompt efficiently and give various example instructions to LLM AIs to optimize how it performs a task in a way we desire

## LangChain examples

In [15]:
llm_openai= OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [10]:
article = """
Paris Saint-Germain soccer superstar Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.

France’s leading sports daily L’Equipe said on its website Sunday that the PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons. No details were given as to how much the transfer fee would be for Neymar, who has also been linked with a return to Barcelona.

The Associated Press has asked defending French champion PSG to confirm the deal. The request was not immediately answered.

Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league on Saturday after training alone on Friday, with the club saying it was because he was recovering from a viral infection. But Neymar is heading for a PSG exit and could leave during the summer transfer window.

The 31-year-old Brazilian joined PSG from Barcelona for a world-record fee of 222 million euros (now $244 million) six years ago, the same year Kylian Mbappe joined from Monaco in a deal worth 180 million euros.

PSG had already received a world-record $332 million bid from Al-Hilal for Mbappe, who is in the last year of his contract and has been mired in a transfer standoff. Mbappe, who reportedly refused to meet with representatives from the Saudi club when they were in Paris last month, has been strongly linked with a move to Real Madrid next season on a free transfer.

Tensions between Mbappe and PSG eased a little on Sunday after he was allowed to return to training following “constructive and positive talks” between the two parties.

Mbappe sat out Saturday’s draw with Lorient after being left out of training. Midfielder Marco Verratti was also not selected by new coach Luis Enrique, and has been linked with a move to the Saudi league after joining PSG in 2012.

"""

In [11]:
len(article)

1823

In [16]:
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template = "Extract the key facts of this text. Don't include opinions. Give fact a number and keep them short sentences. :\n\n {text_input}"
)

In [17]:
# Example of what "PromptTemplate" is doing
fact_extraction_prompt.format(text_input="Doge")

"Extract the key facts of this text. Don't include opinions. Give fact a number and keep them short sentences. :\n\n Doge"

In [18]:
fact_extraction_chain = LLMChain(llm=llm_openai, prompt=fact_extraction_prompt)

In [19]:
facts = fact_extraction_chain.run(article)
facts

'1. Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.\n2. The PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons.\n3. No details were given as to how much the transfer fee would be for Neymar.\n4. Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league after training alone.\n5. Neymar joined PSG from Barcelona for a world-record fee of 222 million euros (now $244 million) six years ago.\n6. PSG had received a world-record $332 million bid from Al-Hilal for Kylian Mbappe.\n7. Mbappe is in the last year of his contract and has been mired in a transfer standoff.\n8. Mbappe reportedly refused to meet with representatives from the Saudi club when they were in Paris last month.\n9. Mbappe has been strongly linked with a move to Real Madrid next season on a free transfer.\n10. Mbappe sat out Saturday’s draw with Lorient after being left out of training.\n11. Midfielder Marco Verratti was als

In [30]:
print(facts)

1. Neymar, a Paris Saint-Germain (PSG) soccer player, has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal.
2. Neymar will receive a total of 160 million euros ($175 million) over two seasons.
3. No details were given about the transfer fee for Neymar.
4. Neymar missed PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection.
5. Neymar joined PSG from Barcelona for a world-record fee of 222 million euros ($244 million) six years ago.
6. PSG received a world-record $332 million bid from Al-Hilal for Kylian Mbappe.
7. Mbappe is in the last year of his contract and has been involved in a transfer standoff.
8. Mbappe reportedly refused to meet with representatives from Al-Hilal.
9. Mbappe returned to training after "constructive and positive talks" with PSG.
10. Marco Verratti, a midfielder, was not selected for Saturday's draw with Lorient and has been linked with a move to the Saudi league.
